In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 17.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 3.8 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp310-cp310-linux_x86_64.whl size=317853 sha256=fa34c5c896e14c1aa17f938b82bf4bdfc8fd87d4cb9161ee567fd81b2b5b7e9f
  Stored in directory: /root/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee


In [3]:
import yfinance as yf # yahoo finance 套件

import pandas as pd # 結構化資料處理套件

import datetime as dt # 時間套件

In [4]:
# 輸入股票代號

stock_id = "2330.TW"

# stock_id = "2357.tw"

# stock_id = "AAPL"

# stock_id = "NVDA"



'''今天往前算幾天'''

# 資料結束時間

#end = dt.date.today()

# output : 2024-09-30

end = dt.date(2024, 12, 5)

# 抓取半年資料（現在-過去180天）

start = end - dt.timedelta(days=300)



'''抓取特定時間'''

# start = dt.datetime(2024, 1, 1)  # 指定的開始日期

# end = dt.datetime(2024, 6, 30)    # 指定的結束日期





# 參數1: 股票代號, 參數2: 開始時間, 參數3: 結束時間

df = yf.download(stock_id, start=start, end=end).reset_index()



df

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,2330.TW,2330.TW,2330.TW,2330.TW,2330.TW,2330.TW
0,2024-02-15,688.021301,697.0,709.0,693.0,709.0,112945296
1,2024-02-16,674.201660,683.0,699.0,683.0,697.0,44232811
2,2024-02-19,669.265991,678.0,682.0,674.0,674.0,34408673
3,2024-02-20,678.150085,687.0,688.0,675.0,675.0,30166461
4,2024-02-21,672.227417,681.0,683.0,678.0,678.0,30564485
...,...,...,...,...,...,...,...
193,2024-11-28,1005.000000,1005.0,1005.0,992.0,1005.0,33689956
194,2024-11-29,996.000000,996.0,1010.0,994.0,995.0,40094983
195,2024-12-02,1035.000000,1035.0,1040.0,1015.0,1020.0,31168404


In [56]:
# 模擬策略的回測（用於計算最終資金）
def backtest_strategy(df, params):
    short_ma, stop_gain, stop_loss = params  # 參數
    
    # 計算移動平均
    df['MA_Short'] = df['Close'].rolling(window=short_ma).mean()
    
    position = 0  # 是否持倉
    buy_price = 0
    equity = 100000  # 初始資金
    initial_equity = equity  # 保存初始本金
    
    for i in range(short_ma, len(df)):  # 避免開盤時沒有足夠的資料
        if position == 0 and pd.notna(df['MA_Short'][i]) and pd.notna(df['Close'][i]):
            # 買入條件：MA_Short > Close
            if df['MA_Short'][i] < df['Close'][i]:
                buy_price = df['Close'][i]
                position = 1  # 持倉
        elif position == 1:
            # 停利停損賣出條件
            profit = (df['Close'][i] - buy_price) / buy_price
            if profit >= stop_gain or profit <= -stop_loss:
                equity *= (1 + profit)
                position = 0  # 清倉

    # 最後強制平倉
    if position == 1:
        final_profit = (df['Close'].iloc[-1] - buy_price) / buy_price
        equity *= (1 + final_profit)

    # 計算回報率
    total_return = (equity - initial_equity) / initial_equity
    return equity, total_return


In [11]:
pip install deap

Note: you may need to restart the kernel to use updated packages.


In [51]:
import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

In [57]:
from deap import base, creator, tools, algorithms
import random

# 定義個體和適應度
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# 創建個體的函數
def create_individual():
    return [random.randint(1, 100), random.uniform(0.01, 0.1), random.uniform(0.01, 0.1)]  # 假設參數有 3 個，隨機生成

# 交配操作：交換基因
def mate(ind1, ind2):
    crossover_point = random.randint(1, len(ind1) - 1)
    ind1[crossover_point:], ind2[crossover_point:] = ind2[crossover_point:], ind1[crossover_point:]
    return ind1, ind2

# 突變操作：隨機改變一個基因的值
def mutate(ind):
    mutation_point = random.randint(0, len(ind) - 1)
    if mutation_point == 0:
        ind[mutation_point] = random.randint(1, 100)
    else:
        ind[mutation_point] = random.uniform(0.01, 0.1)
    return ind,

# 計算適應度：根據回測結果來計算最終資金
def evaluate(individual):
    short_ma, stop_gain, stop_loss = individual  # 取出參數
    equity, total_return = backtest_strategy(df, individual)  # 回測並獲取最終資金與回報率
    return total_return,  # 這裡以回報率作為適應度

# 註冊工具箱
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", mate)  # 註冊交配操作
toolbox.register("mutate", mutate)  # 註冊突變操作
toolbox.register("evaluate", evaluate)  # 註冊適應度計算操作
toolbox.register("select", tools.selTournament, tournsize=3)  # 註冊選擇操作

# 進行基因演算法
def run_ga(df):
    population = toolbox.population(n=10)  # 初始化族群
    CXPB, MUTPB, NGEN = 0.7, 0.2, 40  # 交配機率，突變機率，演化代數

    for gen in range(NGEN):
        offspring = toolbox.select(population, len(population))  # 選擇操作
        offspring = list(map(toolbox.clone, offspring))  # 複製族群

        # 交配操作
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)  # 交配
                del child1.fitness.values, child2.fitness.values  # 刪除原來的適應度

        # 突變操作
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)  # 突變
                del mutant.fitness.values  # 刪除原來的適應度

        # 計算適應度
        for ind in offspring:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # 取代族群
        population[:] = offspring

    best_individual = tools.selBest(population, 1)[0]
    return best_individual

# 假設 df 已經是包含股票數據的 DataFrame
best_params = run_ga(df)
print("最佳參數:", best_params)

最佳參數: [20, 0.025608279801537154, 0.07055326834223985]


In [58]:
# 假設df是你使用的股市數據
best_params = [20, 0.025608279801537154, 0.07055326834223985]  # 從GA中得到的最佳參數

# 進行回測
short_ma, stop_gain, stop_loss = best_params

# 回測策略
equity, total_return = backtest_strategy(df, best_params)

# 輸出回測結果
print(f"回測結果:")
print(f"最終資金: {equity:.2f}")
print(f"回報率: {total_return:.2%}")


回測結果:
最終資金: 100000.00
回報率: 0.00%
